In [0]:
%load_ext autoreload
%autoreload 2


In [0]:
# carregamento de bibliotecas
import sys
import os
import ipeadatapy as ipea

root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.utils import *


In [0]:
 dbutils.library.restartPython()

In [0]:
# construção da tabela histórica do preço de petróleo bruto
GASOLINA = "ANP_PRGASOL"
TABELA_HISTORICA = "`api-forecasting`.bronze.precos_gasolina_historico_raw"

historico = ipea.timeseries(GASOLINA)

# filtragem da base histórica para o período dos dados
historico = historico.loc["2015-01-01":"2025-11-19"]

# transformação do dataset no formato spark
historico = criar_tabela_spark(df=historico)
historico.show()



In [0]:
# sanitizacao dos nomes das colunas para persistência como delta
df_limpo = historico.withColumnRenamed("VALUE (R$)", "VALUE_REAIS") \
                    .withColumnRenamed("RAW DATE", "RAW_DATE")
          

In [0]:
# visualização da tabela sanitizada
df_limpo.show(5)

In [0]:
# persistência da tabela na camada bronze
df = salvar_tabela_delta_spark(df_limpo, 
                          mode='overwrite',
                          merge_schema=False,
                          partitionby=['RAW_DATE'], 
                          path=TABELA_HISTORICA)

In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.precos_gasolina_historico_raw
LIMIT 10